Our problem is an image recognition problem, to identify digits from a given 28 x 28 image. We have a subset of images for training and the rest for testing our model. So first, download the train and test files. 

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import GridSearchCV

import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K

In [28]:
# To stop potential randomness
seed = 128

In [29]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [30]:
#to increase the speed of calculations look only at 2s and 7s
train_picks = np.logical_or(y_train==2,y_train==7)
test_picks = np.logical_or(y_test==2,y_test==7)

x_train = x_train[train_picks]
x_test = x_test[test_picks]
y_train = np.array(y_train[train_picks]==7,dtype=int)
y_test = np.array(y_test[test_picks]==7,dtype=int)

CNN converg faster on [0..1] data than on [0..255].

In [31]:
# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [32]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

In [33]:
# Encode labels to one hot vectors 
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)

We want to build simple CNN models to classify the MNIST dataset and uses sklearn's GridSearchCV to find the best hyperparameter model

In [13]:
##### BATCH SIZE #####

def create_model():
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation='softmax'))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    
    return model


np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [6],
              'batch_size': [10, 20, 40, 60, 80, 100]}

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Epoch 1/6
8148/8148 [==============================] - 5s 601us/step - loss: 0.0549 - acc: 0.9821
Epoch 2/6
8148/8148 [==============================] - 4s 432us/step - loss: 0.0298 - acc: 0.9914
Epoch 3/6
8148/8148 [==============================] - 4s 431us/step - loss: 0.0198 - acc: 0.9941
Epoch 4/6
8148/8148 [==============================] - 4s 431us/step - loss: 0.0142 - acc: 0.9962
Epoch 5/6
8148/8148 [==============================] - 4s 468us/step - loss: 0.0109 - acc: 0.9977
Epoch 6/6
8148/8148 [==============================] - 1s 99us/step
Epoch 1/6
8149/8149 [==============================] - 4s 488us/step - loss: 0.0531 - acc: 0.9844
Epoch 2/6
8149/8149 [==============================] - 4s 449us/step - loss: 0.0233 - acc: 0.9934
Epoch 3/6
8149/8149 [==============================] - 4s 439us/step - loss: 0.0154 - acc: 0.9945
Epoch 4/6
8149/8149 [==============================] - 4s 446us/step - loss: 0.0110 - ac

Epoch 5/6
8149/8149 [==============================] - 1s 98us/step - loss: 0.0121 - acc: 0.9969
Epoch 6/6
8149/8149 [==============================] - 0s 34us/step
Epoch 1/6
8148/8148 [==============================] - 1s 167us/step - loss: 0.0854 - acc: 0.9702
Epoch 2/6
8148/8148 [==============================] - 1s 77us/step - loss: 0.0354 - acc: 0.9888
Epoch 3/6
8148/8148 [==============================] - 1s 79us/step - loss: 0.0251 - acc: 0.9930
Epoch 4/6
8148/8148 [==============================] - 1s 77us/step - loss: 0.0175 - acc: 0.9953
Epoch 5/6
8148/8148 [==============================] - 1s 77us/step - loss: 0.0139 - acc: 0.9959
Epoch 6/6
8148/8148 [==============================] - 0s 29us/step
Epoch 1/6
8149/8149 [==============================] - 1s 173us/step - loss: 0.0787 - acc: 0.9726
Epoch 2/6
8149/8149 [==============================] - 1s 81us/step - loss: 0.0298 - acc: 0.9896
Epoch 3/6
8149/8149 [==============================] - 1s 76us/step - loss: 0.0187 - a

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  3.2min finished


Epoch 1/6
12223/12223 [==============================] - 7s 543us/step - loss: 0.0476 - acc: 0.9840
Epoch 2/6
12223/12223 [==============================] - 6s 478us/step - loss: 0.0232 - acc: 0.9930
Epoch 3/6
12223/12223 [==============================] - 6s 465us/step - loss: 0.0164 - acc: 0.9961
Epoch 4/6
12223/12223 [==============================] - 6s 466us/step - loss: 0.0137 - acc: 0.9971
Epoch 5/6
12223/12223 [==============================] - 6s 470us/step - loss: 0.0107 - acc: 0.9973
Epoch 6/6
12223/12223 [==============================] - 6s 489us/step - loss: 0.0091 - acc: 0.9980
Best: 0.992800 using {'batch_size': 10, 'epochs': 6}
0.992800 (0.001335) with: {'batch_size': 10, 'epochs': 6}
0.992228 (0.001478) with: {'batch_size': 20, 'epochs': 6}
0.992637 (0.001591) with: {'batch_size': 40, 'epochs': 6}
0.992555 (0.000989) with: {'batch_size': 60, 'epochs': 6}
0.992637 (0.001219) with: {'batch_size': 80, 'epochs': 6}
0.991491 (0.001104) with: {'batch_size': 100, 'epochs': 6

In [14]:
##### OPTIMIZATION ALGORITHM #####

def create_model(optimizer):
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation='softmax'))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [6],
              'batch_size': [10],
              'optimizer': ['SGD', 'Adam']
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Epoch 1/6
8148/8148 [==============================] - 6s 694us/step - loss: 0.0551 - acc: 0.9823
Epoch 2/6
8148/8148 [==============================] - 5s 556us/step - loss: 0.0299 - acc: 0.9914
Epoch 3/6
8148/8148 [==============================] - 5s 635us/step - loss: 0.0198 - acc: 0.9942
Epoch 4/6
8148/8148 [==============================] - 5s 618us/step - loss: 0.0143 - acc: 0.9964
Epoch 5/6
8148/8148 [==============================] - 5s 591us/step - loss: 0.0111 - acc: 0.9975
Epoch 6/6
8148/8148 [==============================] - 1s 142us/step
Epoch 1/6
8149/8149 [==============================] - 5s 628us/step - loss: 0.0532 - acc: 0.9843
Epoch 2/6
8149/8149 [==============================] - 4s 475us/step - loss: 0.0233 - acc: 0.9931
Epoch 3/6
8149/8149 [==============================] - 4s 524us/step - loss: 0.0156 - acc: 0.9945
Epoch 4/6
8149/8149 [==============================] - 5s 626us/step - loss: 0.0114 - ac

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.9min finished


Epoch 1/6
12223/12223 [==============================] - 8s 630us/step - loss: 0.0480 - acc: 0.9832
Epoch 2/6
12223/12223 [==============================] - 6s 491us/step - loss: 0.0222 - acc: 0.9937
Epoch 3/6
12223/12223 [==============================] - 6s 500us/step - loss: 0.0167 - acc: 0.9957
Epoch 4/6
12223/12223 [==============================] - 6s 503us/step - loss: 0.0131 - acc: 0.9965
Epoch 5/6
12223/12223 [==============================] - 6s 531us/step - loss: 0.0107 - acc: 0.9975
Epoch 6/6
12223/12223 [==============================] - 6s 508us/step - loss: 0.0096 - acc: 0.9980
Best: 0.993210 using {'batch_size': 10, 'epochs': 6, 'optimizer': 'SGD'}
0.993210 (0.001633) with: {'batch_size': 10, 'epochs': 6, 'optimizer': 'SGD'}
0.992964 (0.001728) with: {'batch_size': 10, 'epochs': 6, 'optimizer': 'Adam'}
Best: 0.993210 using {'batch_size': 10, 'epochs': 6, 'optimizer': 'SGD'}
2060/2060 [==============================] - 1s 475us/step
Test Accuracy 0.9917475711373449


In [21]:
##### LOSS FUNCTION #####

def create_model(optimizer, loss):
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation='softmax'))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution        

    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {
              'epochs': [6],
              'batch_size': [10],
              'optimizer': ['SGD'],
              'loss': ['categorical_crossentropy', 'binary_crossentropy']
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Epoch 1/6
8148/8148 [==============================] - 5s 596us/step - loss: 0.1111 - acc: 0.9672
Epoch 2/6
8148/8148 [==============================] - 3s 311us/step - loss: 0.0479 - acc: 0.9850
Epoch 3/6
8148/8148 [==============================] - 3s 364us/step - loss: 0.0393 - acc: 0.9881
Epoch 4/6
8148/8148 [==============================] - 3s 327us/step - loss: 0.0339 - acc: 0.9905
Epoch 5/6
8148/8148 [==============================] - 3s 363us/step - loss: 0.0299 - acc: 0.9913
Epoch 6/6
8148/8148 [==============================] - 1s 157us/step
Epoch 1/6
8149/8149 [==============================] - 4s 523us/step - loss: 0.1087 - acc: 0.9694
Epoch 2/6
8149/8149 [==============================] - 3s 380us/step - loss: 0.0460 - acc: 0.9848
Epoch 3/6
8149/8149 [==============================] - 3s 342us/step - loss: 0.0369 - acc: 0.9881
Epoch 4/6
8149/8149 [==============================] - 3s 340us/step - loss: 0.0317 - ac

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.3min finished


Epoch 1/6
12223/12223 [==============================] - 6s 506us/step - loss: 0.0829 - acc: 0.9748
Epoch 2/6
12223/12223 [==============================] - 4s 360us/step - loss: 0.0404 - acc: 0.9883
Epoch 3/6
12223/12223 [==============================] - 4s 354us/step - loss: 0.0340 - acc: 0.9885
Epoch 4/6
12223/12223 [==============================] - 4s 348us/step - loss: 0.0293 - acc: 0.9912
Epoch 5/6
12223/12223 [==============================] - 4s 353us/step - loss: 0.0256 - acc: 0.9923
Epoch 6/6
12223/12223 [==============================] - 4s 350us/step - loss: 0.0223 - acc: 0.9935
Best: 0.989282 using {'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.988464 (0.000696) with: {'batch_size': 10, 'epochs': 6, 'loss': 'categorical_crossentropy', 'optimizer': 'SGD'}
0.989282 (0.001335) with: {'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
Best: 0.989282 using {'batch_size': 10, 'epochs': 6, 'loss': 'binary_cros

In [24]:
##### NEURON ACTIVATION FUNCTION 1 #####

def create_model(optimizer, loss, activation_1, activation_2, activation_3):
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation=activation_1)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation=activation_2)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation=activation_3))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution

    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [6],
              'batch_size': [10],
              'optimizer': ['SGD'],
              'loss': ['binary_crossentropy'],
              'activation_1': ['softmax', 'relu', 'tanh', 'sigmoid'],
              'activation_2': ['relu'],
              'activation_3': ['softmax']
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Epoch 1/6
8148/8148 [==============================] - 7s 853us/step - loss: 0.6896 - acc: 0.5647
Epoch 2/6
8148/8148 [==============================] - 3s 397us/step - loss: 0.6829 - acc: 0.6183
Epoch 3/6
8148/8148 [==============================] - 3s 399us/step - loss: 0.4644 - acc: 0.9001
Epoch 4/6
8148/8148 [==============================] - 4s 448us/step - loss: 0.1580 - acc: 0.9572
Epoch 5/6
8148/8148 [==============================] - 4s 444us/step - loss: 0.1045 - acc: 0.9676
Epoch 6/6
8148/8148 [==============================] - 2s 226us/step
Epoch 1/6
8149/8149 [==============================] - 7s 829us/step - loss: 0.6903 - acc: 0.5291
Epoch 2/6
8149/8149 [==============================] - 3s 375us/step - loss: 0.6854 - acc: 0.5559
Epoch 3/6
8149/8149 [==============================] - 3s 374us/step - loss: 0.6666 - acc: 0.7033
Epoch 4/6
8149/8149 [==============================] - 3s 377us/step - loss: 0.3045 - a

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.4min finished


Epoch 1/6
12223/12223 [==============================] - 10s 783us/step - loss: 0.0950 - acc: 0.9701
Epoch 2/6
12223/12223 [==============================] - 5s 415us/step - loss: 0.0419 - acc: 0.9866
Epoch 3/6
12223/12223 [==============================] - 5s 416us/step - loss: 0.0347 - acc: 0.9885
Epoch 4/6
12223/12223 [==============================] - 5s 414us/step - loss: 0.0297 - acc: 0.9905
Epoch 5/6
12223/12223 [==============================] - 5s 416us/step - loss: 0.0260 - acc: 0.9925
Epoch 6/6
12223/12223 [==============================] - 5s 418us/step - loss: 0.0226 - acc: 0.9933
Best: 0.989282 using {'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.977747 (0.002806) with: {'activation_1': 'softmax', 'activation_2': 'relu', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.989282 (0.001335) with: {'activ

In [25]:
##### NEURON ACTIVATION FUNCTION 2 #####

def create_model(optimizer, loss, activation_1, activation_2, activation_3):
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation=activation_1)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation=activation_2)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation=activation_3))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution


    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [6],
              'batch_size': [10],
              'optimizer': ['SGD'],
              'loss': ['binary_crossentropy'],
              'activation_1': ['relu'],
              'activation_2': ['softmax', 'relu', 'tanh', 'sigmoid'],
              'activation_3': ['softmax']
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Epoch 1/6
8148/8148 [==============================] - 9s 1ms/step - loss: 0.6930 - acc: 0.5147
Epoch 2/6
8148/8148 [==============================] - 3s 419us/step - loss: 0.6907 - acc: 0.5459
Epoch 3/6
8148/8148 [==============================] - 4s 438us/step - loss: 0.6877 - acc: 0.5611
Epoch 4/6
8148/8148 [==============================] - 4s 437us/step - loss: 0.6822 - acc: 0.6632
Epoch 5/6
8148/8148 [==============================] - 4s 435us/step - loss: 0.6022 - acc: 0.9014
Epoch 6/6
8148/8148 [==============================] - 2s 271us/step
Epoch 1/6
8149/8149 [==============================] - 8s 989us/step - loss: 0.6919 - acc: 0.5192
Epoch 2/6
8149/8149 [==============================] - 3s 419us/step - loss: 0.6896 - acc: 0.5180
Epoch 3/6
8149/8149 [==============================] - 3s 414us/step - loss: 0.6869 - acc: 0.5177
Epoch 4/6
8149/8149 [==============================] - 3s 418us/step - loss: 0.6819 - acc

Epoch 5/6
8149/8149 [==============================] - 4s 436us/step - loss: 0.0466 - acc: 0.9853
Epoch 6/6
8149/8149 [==============================] - 2s 288us/step


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.5min finished


Epoch 1/6
12223/12223 [==============================] - 13s 1ms/step - loss: 0.0950 - acc: 0.9701
Epoch 2/6
12223/12223 [==============================] - 6s 511us/step - loss: 0.0419 - acc: 0.9866
Epoch 3/6
12223/12223 [==============================] - 5s 448us/step - loss: 0.0347 - acc: 0.9885
Epoch 4/6
12223/12223 [==============================] - 5s 446us/step - loss: 0.0297 - acc: 0.9905
Epoch 5/6
12223/12223 [==============================] - 5s 446us/step - loss: 0.0261 - acc: 0.9925
Epoch 6/6
12223/12223 [==============================] - 5s 449us/step - loss: 0.0226 - acc: 0.9931
Best: 0.989364 using {'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.972347 (0.008485) with: {'activation_1': 'relu', 'activation_2': 'softmax', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.989364 (0.001423) with: {'activat

In [26]:
##### NEURON ACTIVATION FUNCTION 3 #####

def create_model(optimizer, loss, activation_1, activation_2, activation_3):
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation=activation_1)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation=activation_2)) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation=activation_3))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution

    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [6],
              'batch_size': [10],
              'optimizer': ['SGD'],
              'loss': ['binary_crossentropy'],
              'activation_1': ['relu'],
              'activation_2': ['relu'],
              'activation_3': ['softmax', 'relu', 'tanh', 'sigmoid']
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print("Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Epoch 1/6
8148/8148 [==============================] - 9s 1ms/step - loss: 0.1111 - acc: 0.9674
Epoch 2/6
8148/8148 [==============================] - 4s 452us/step - loss: 0.0479 - acc: 0.9851
Epoch 3/6
8148/8148 [==============================] - 4s 470us/step - loss: 0.0393 - acc: 0.9881
Epoch 4/6
8148/8148 [==============================] - 4s 489us/step - loss: 0.0339 - acc: 0.9907
Epoch 5/6
8148/8148 [==============================] - 4s 466us/step - loss: 0.0299 - acc: 0.9913
Epoch 6/6
8148/8148 [==============================] - 2s 284us/step
Epoch 1/6
8149/8149 [==============================] - 9s 1ms/step - loss: 0.1087 - acc: 0.9694
Epoch 2/6
8149/8149 [==============================] - 4s 449us/step - loss: 0.0460 - acc: 0.9848
Epoch 3/6
8149/8149 [==============================] - 4s 444us/step - loss: 0.0369 - acc: 0.9881
Epoch 4/6
8149/8149 [==============================] - 4s 449us/step - loss: 0.0317 - acc: 

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  7.1min finished


Epoch 1/6
12223/12223 [==============================] - 12s 988us/step - loss: 0.0950 - acc: 0.9703
Epoch 2/6
12223/12223 [==============================] - 6s 457us/step - loss: 0.0419 - acc: 0.9866
Epoch 3/6
12223/12223 [==============================] - 6s 456us/step - loss: 0.0347 - acc: 0.9886
Epoch 4/6
12223/12223 [==============================] - 6s 467us/step - loss: 0.0297 - acc: 0.9905
Epoch 5/6
12223/12223 [==============================] - 5s 406us/step - loss: 0.0260 - acc: 0.9924
Epoch 6/6
12223/12223 [==============================] - 5s 423us/step - loss: 0.0226 - acc: 0.9932
Best: 0.988464 using {'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.988464 (0.000696) with: {'activation_1': 'relu', 'activation_2': 'relu', 'activation_3': 'softmax', 'batch_size': 10, 'epochs': 6, 'loss': 'binary_crossentropy', 'optimizer': 'SGD'}
0.369631 (0.265103) with: {'activati

In [34]:
##### Final Model #####
##### Epochs - The more the better usually. #####

def create_model():
    
    model = Sequential()
    # a basic feed-forward model
    model.add(Flatten()) 
    # takes our 28x28 and makes it 1x784
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(128, activation='relu')) 
    # a simple fully-connected layer, 128 units, relu activation
    model.add(Dense(2, activation='softmax'))  
    # our output layer. 2 units for 2 classes. Softmax for probability distribution

    model.compile(loss=keras.losses.binary_crossentropy, 
                  optimizer=keras.optimizers.SGD(), 
                  metrics=['accuracy'])
    
    return model

# fix random seed for reproducibility
np.random.seed(seed)

# Test all combinations of the following parameters:
param_grid = {'epochs': [20],
              'batch_size': [20, 40, 60]
              }

my_classifier = KerasClassifier(create_model)

grid = GridSearchCV(my_classifier, param_grid, cv=3, n_jobs=1, verbose=1)

grid_result = grid.fit(x_train, y_train)

# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
# Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Print Test Accuracy
print(" Test Accuracy", grid_result.score(x_test, y_test))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Epoch 1/20
8148/8148 [==============================] - 9s 1ms/step - loss: 0.1572 - acc: 0.9566
Epoch 2/20
8148/8148 [==============================] - 2s 247us/step - loss: 0.0610 - acc: 0.9806
Epoch 3/20
8148/8148 [==============================] - 2s 243us/step - loss: 0.0492 - acc: 0.9842
Epoch 4/20
8148/8148 [==============================] - 2s 242us/step - loss: 0.0432 - acc: 0.9871
Epoch 5/20
8148/8148 [==============================] - 2s 246us/step - loss: 0.0391 - acc: 0.9886
Epoch 6/20
8148/8148 [==============================] - 2s 244us/step - loss: 0.0360 - acc: 0.9892
Epoch 7/20
8148/8148 [==============================] - 2s 244us/step - loss: 0.0335 - acc: 0.9905
Epoch 8/20
8148/8148 [==============================] - 2s 242us/step - loss: 0.0310 - acc: 0.9912
Epoch 9/20
8148/8148 [==============================] - 2s 246us/step - loss: 0.0294 - acc: 0.9908
Epoch 10/20
8148/8148 [=============================

8149/8149 [==============================] - 1s 105us/step - loss: 0.0324 - acc: 0.9903
Epoch 17/20
8149/8149 [==============================] - 1s 103us/step - loss: 0.0316 - acc: 0.9902
Epoch 18/20
8149/8149 [==============================] - 1s 102us/step - loss: 0.0308 - acc: 0.9910
Epoch 19/20
8149/8149 [==============================] - 1s 101us/step - loss: 0.0299 - acc: 0.9913
Epoch 20/20
8149/8149 [==============================] - 1s 66us/step
Epoch 1/20
8149/8149 [==============================] - 8s 962us/step - loss: 0.2581 - acc: 0.9326
Epoch 2/20
8149/8149 [==============================] - 1s 99us/step - loss: 0.0965 - acc: 0.9707
Epoch 3/20
8149/8149 [==============================] - 1s 97us/step - loss: 0.0746 - acc: 0.9752
Epoch 4/20
8149/8149 [==============================] - 1s 99us/step - loss: 0.0641 - acc: 0.9790
Epoch 5/20
8149/8149 [==============================] - 1s 97us/step - loss: 0.0576 - acc: 0.9809
Epoch 6/20
8149/8149 [=============================

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  5.8min finished


Epoch 1/20
12223/12223 [==============================] - 10s 838us/step - loss: 0.1296 - acc: 0.9651
Epoch 2/20
12223/12223 [==============================] - 3s 250us/step - loss: 0.0524 - acc: 0.9837
Epoch 3/20
12223/12223 [==============================] - 3s 249us/step - loss: 0.0432 - acc: 0.9872
Epoch 4/20
12223/12223 [==============================] - 3s 248us/step - loss: 0.0385 - acc: 0.9884
Epoch 5/20
12223/12223 [==============================] - 3s 247us/step - loss: 0.0352 - acc: 0.9892
Epoch 6/20
12223/12223 [==============================] - 3s 251us/step - loss: 0.0322 - acc: 0.9899
Epoch 7/20
12223/12223 [==============================] - 3s 248us/step - loss: 0.0297 - acc: 0.9903
Epoch 8/20
12223/12223 [==============================] - 3s 248us/step - loss: 0.0279 - acc: 0.9912
Epoch 9/20
12223/12223 [==============================] - 3s 250us/step - loss: 0.0258 - acc: 0.9927
Epoch 10/20
12223/12223 [==============================] - 3s 253us/step - loss: 0.0246 - 